## Needleman-Wunsch

Needleman-Wunsch is the most basic global alignment algorithm. It uses a dynamic programming approach to find all alignments between two sequences which are optimal with respect to some pre-determined scores for matches, mismatches, and gaps. Needleman-Wunsch attempts to maximize the alignment score by efficiently testing all possible alignments (in so far as testing all possible alignments can be efficient, in this case $O(n^3)$).

The basic version originally described by its authors uses the simple scoring technique described above. More advanced techniques use a mismatch matrix to assign different scores depending on which residues are being compared; for example, we might want a mismatch between G and C to have a lower penalty than between G and A. The simple implementation below opts to use the original naive system.

### Code

In [3]:
DEFAULT_GAP = -6
DEFAULT_MATCH = 5
DEFAULT_MISMATCH = -2

In [4]:
def NW_makeTable(X, Y, g):
    ''' Given 2 sequences and the gap penalty, construct the Needleman-Wunsch starting table.
    
    Args:
        X (str): The first sequence.
        Y (str): The second sequence.
        g (int): The gap penalty.
    
    Returns:
        list: A len(X)+1 by len(Y)+1 table where all positions are 0 except the first row and column,
        which are filled linearly with the gap penalty.
    
    '''
    table = []
    for i in range(0, len(X)+1):
        table.append([0]*(len(Y)+1))
    for i in range(0,len(X)+1,1):
        table[i][0] = i*g
    for j in range(0,len(Y)+1,1):
        table[0][j] = j*g

    return table


def printTable(table, X, Y):
    ''' Print out a scoring table and its two sequences in a nice format.
    
    Args:
        table (list): The Needleman-Wunsch scoring table.
        X (str): The first sequence, with sentinal character.
        Y (str): The second sequence, with sentinal character.
    '''
    pt = ' '

    for y in Y:
        pt += y.rjust(4)
    pt += '\n'
    for i in range(0, len(table)):
        prow = X[i]
        for c in table[i]:
            prow += str(c).rjust(4)
        prow += '\n'
        pt += prow
    print pt


def NW_traceBack(table, X, Y, g, m, n, i, j, a1, a2, results):
    ''' Recursively traceback a scoring table to construct the alignments.
    
    Args:
        table (list): The completed scoring table.
        X (str): The first sequence, with sentinal.
        Y (str): The second sequence, with sentinal.
        g (int): The gap penalty.
        m (int): The match score.
        n (int): The mismatch penalty.
        i (int): The index in X.
        j (int): The index in Y.
        a1 (str): The alignment string for X.
        a2 (str): The alignment string for Y.
        results (list): List object to place results in.
    '''
    
    if i < 0 or j < 0:
        raise IndexError('Tracback went out of bounds ({i},{j})'.format(i=i, j=j))
    if i == 0 and j != 0:
        NW_traceBack(table, X, Y, g, m, n, i, j-1, a1+'-', a2+Y[j], results)
    if j == 0 and i != 0:
        NW_traceBack(table, X, Y, g, m, n, i-1, j, a1+X[i], a2+'-', results)
    if i == 0 and j == 0:
        a1 = a1[::-1]
        a2 = a2[::-1]
        results.append((a1, a2))
    else:
        a = table[i][j]
        b = table[i-1][j]
        c = table[i][j-1]
        d = table[i-1][j-1]
        if a-d == m and X[i] == Y[j]:
            NW_traceBack(table, X, Y, g, m, n, i-1, j-1, a1+X[i], a2+Y[j], results)
        if a-d == n and X[i] != Y[j]:
            NW_traceBack(table, X, Y, g, m, n, i-1, j-1, a1+X[i], a2+Y[j], results)
        if a - b == g:
            NW_traceBack(table, X, Y, g, m, n, i-1, j, a1+X[i], a2+'-', results)
        if a - c == g:
            NW_traceBack(table, X, Y, g, m, n, i, j-1, a1+'-', a2+Y[j], results)

In [5]:
def NeedlemanWunsch(X, Y, gap, match, mismatch, quiet=True):
    ''' Perform a global alignment between the sequences X and Y using the Needleman-Wunsch algorithm.
    
    Args:
        X (str): The first sequence to be aligned.
        Y (str): The second sequence to be aligned.
        gap (int): The gap penalty.
        match (int): The match score.
        mismatch (int): The mismatch penalty.
        quiet (Optional[bool]): Whether or not to print out diagnostic information. Defaults to True.
        
    Returns:
        list: A list of the optimal alignments.
    '''
    table = NW_makeTable(X, Y, gap)
    
    if not quiet:
        print "===Needleman-Wunsch Global Alignment==="
        print "Table Initialized"
        
    X = '*' + X
    Y = '*' + Y
    
    if not quiet:
        printTable(table, X, Y)
    
    score = lambda a, b: match if a == b else mismatch

    for i in range(1, len(table)):
        for j in range(1, len(table[i])):
            a = table[i-1][j-1] + score(X[i], Y[j])
            b = table[i-1][j] + gap
            c = table[i][j-1] + gap
            table[i][j] = max(a, b, c)
    
    if not quiet:
        print "Completed Table"
        printTable(table, X, Y)

        print "Optimal Global Alignments:\n"

    results = []
    NW_traceBack(table, X, Y, gap, match, mismatch, len(X)-2, len(Y)-2, '', '', results)
    if not quiet:
        for a1, a2 in results:
            print a1, '\n', a2, '\n'
        print "--End of Optimal Alignment List--"

    if not quiet:
        print "Parameters: gap={} match={} mismatch={}".format(gap, match, mismatch)
        print "Optimal Global Alignment Score: {}".format(table[len(table)-1][len(table[0])-1])
        print "=" * 60 + '\n'
    
    return results

### Demonstration

In [6]:
NeedlemanWunsch('ATTCGGCT', 'AGTTGGGCCCGCGT', DEFAULT_GAP, DEFAULT_MATCH, DEFAULT_MISMATCH, quiet=True)

[('A-TT----CGGC-', 'AGTTGGGCCCGCG'),
 ('A-TT---C-GGC-', 'AGTTGGGCCCGCG'),
 ('A-TT---CG-GC-', 'AGTTGGGCCCGCG'),
 ('A-TT-CG---GC-', 'AGTTGGGCCCGCG'),
 ('A-TTC-G---GC-', 'AGTTGGGCCCGCG'),
 ('A-TTCG----GC-', 'AGTTGGGCCCGCG'),
 ('A-TTCGG----C-', 'AGTTGGGCCCGCG'),
 ('A-TTCGG--C---', 'AGTTGGGCCCGCG'),
 ('A-TTCGG-C----', 'AGTTGGGCCCGCG'),
 ('A-TTCGGC-----', 'AGTTGGGCCCGCG')]

In [7]:
NeedlemanWunsch('TATATATATA', 'GCATAAGGT', DEFAULT_GAP, DEFAULT_MATCH, DEFAULT_MISMATCH, quiet=False)

===Needleman-Wunsch Global Alignment===
Table Initialized
    *   G   C   A   T   A   A   G   G   T
*   0  -6 -12 -18 -24 -30 -36 -42 -48 -54
T  -6   0   0   0   0   0   0   0   0   0
A -12   0   0   0   0   0   0   0   0   0
T -18   0   0   0   0   0   0   0   0   0
A -24   0   0   0   0   0   0   0   0   0
T -30   0   0   0   0   0   0   0   0   0
A -36   0   0   0   0   0   0   0   0   0
T -42   0   0   0   0   0   0   0   0   0
A -48   0   0   0   0   0   0   0   0   0
T -54   0   0   0   0   0   0   0   0   0
A -60   0   0   0   0   0   0   0   0   0

Completed Table
    *   G   C   A   T   A   A   G   G   T
*   0  -6 -12 -18 -24 -30 -36 -42 -48 -54
T  -6  -2  -8 -14 -13 -19 -25 -31 -37 -43
A -12  -8  -4  -3  -9  -8 -14 -20 -26 -32
T -18 -14 -10  -6   2  -4 -10 -16 -22 -21
A -24 -20 -16  -5  -4   7   1  -5 -11 -17
T -30 -26 -22 -11   0   1   5  -1  -7  -6
A -36 -32 -28 -17  -6   5   6   3  -3  -9
T -42 -38 -34 -23 -12  -1   3   4   1   2
A -48 -44 -40 -29 -18  -7   4   1   2  -1
T

[('TATATATAT', '-GCATAAGG'),
 ('TATATATAT', '-GCATAAGG'),
 ('TATATATAT', 'G-CATAAGG'),
 ('TATATATAT', 'GC-ATAAGG')]

## Smith-Waterman

Smith-Waterman is the companion algorithm to Needleman-Wunsch. However, while the latter performs a global alignment, the former performs a local alignment, by making a few tweaks to the dynamic programming matrix and backtracing. Here, we set the gap row and column to zero, and terminate the backtrace if we reach a zero score.

### Code

In [8]:
def SW_makeTable(X, Y):
    ''' Given 2 sequences, construct the Smith-Waterman starting table.
    
    Args:
        X (string): The first sequence.
        Y (string): The second sequence.
    
    Returns:
        list: A len(X)+1 by len(Y)+1 table where all positions are 0.
    
    '''
    table = []
    for i in range(0, len(X)+1):
        table.append([0]*(len(Y)+1))

    return table

In [9]:
def SW_traceBack(table, X, Y, g, m, n, i, j, a1, a2, results):
    ''' Recursively traceback a Smith-Waterman scoring table to construct the alignments.
    
    Args:
        table (list): The completed scoring table.
        X (string): The first sequence, with sentinal.
        Y (string): The second sequence, with sentinal.
        g (int): The gap penalty.
        m (int): The match score.
        n (int): The mismatch penalty.
        i (int): The index in X.
        j (int): The index in Y.
        a1 (str): The alignment string for X.
        a2 (str): The alignment string for Y.
        results (list): List object to place results in.
    '''
    
    if i < 0 or j < 0:
        raise IndexError('Tracback went out of bounds ({i},{j})'.format(i=i, j=j))

    if i == 0 or j == 0:
        a1 = a1[::-1]
        a2 = a2[::-1]
        results.append((a1, a2))
    else:
        a = table[i][j]
        b = table[i-1][j]
        c = table[i][j-1]
        d = table[i-1][j-1]
        if a-d == m and X[i] == Y[j]:
            SW_traceBack(table, X, Y, g, m, n, i-1, j-1, a1+X[i], a2+Y[j], results)
        if a-d == n and X[i] != Y[j]:
            SW_traceBack(table, X, Y, g, m, n, i-1, j-1, a1+X[i], a2+Y[j], results)
        if a - b == g:
            SW_traceBack(table, X, Y, g, m, n, i-1, j, a1+X[i], a2+'-', results)
        if a - c == g:
            SW_traceBack(table, X, Y, g, m, n, i, j-1, a1+'-', a2+Y[j], results)

In [10]:
def SmithWaterman(X, Y, gap, match, mismatch, quiet=True):
    ''' Perform a global alignment between the sequences X and Y using the Smith-Waterman algorithm.
    
    Args:
        X (str): The first sequence to be aligned.
        Y (str): The second sequence to be aligned.
        gap (int): The gap penalty.
        match (int): The match score.
        mismatch (int): The mismatch penalty.
        quiet (Optional[bool]): Whether or not to print out diagnostic information. Defaults to True.
        
    Returns:
        list: A list of the optimal alignments.
    '''
    table = SW_makeTable(X, Y)
    
    if not quiet:
        print "===Smith-Waterman Local Alignment==="
        print "Table Initialized"
        
    X = '*' + X
    Y = '*' + Y
    
    if not quiet:
        printTable(table, X, Y)
    
    score = lambda a, b: match if a == b else mismatch

    for i in range(1, len(table)):
        for j in range(1, len(table[i])):
            a = table[i-1][j-1] + score(X[i], Y[j])
            b = table[i-1][j] + gap
            c = table[i][j-1] + gap
            table[i][j] = max(a, b, c)
    
    if not quiet:
        print "Completed Table"
        printTable(table, X, Y)

        print "Optimal Local Alignments:\n"
    
    # Smith-Waterman traceback starts at the highest-scoring position. Find it!
    max_pos = (0,0)
    max_val = 0
    for i in xrange(1, len(table)):
        for j in xrange(1, len(table[i])):
            if table[i][j] > max_val:
                max_pos = (i, j)
                max_val = table[i][j]
    results = []
    SW_traceBack(table, X, Y, gap, match, mismatch, max_pos[0], max_pos[1], '', '', results)
    if not quiet:
        for a1, a2 in results:
            print a1, '\n', a2, '\n'
        print "--End of Optimal Alignment List--"

    if not quiet:
        print "Parameters: gap={} match={} mismatch={}".format(gap, match, mismatch)
        print "Optimal Local Alignment Score: {}".format(table[len(table)-1][len(table[0])-1])
        print "=" * 60 + '\n'
    
    return results

### Demonstration

In [11]:
SmithWaterman('AAATTGGGCGCCGGGTTTAAA', 'GGCCCGGGAAATTTA', DEFAULT_GAP, DEFAULT_MATCH, DEFAULT_MISMATCH, quiet=False)

===Smith-Waterman Local Alignment===
Table Initialized
    *   G   G   C   C   C   G   G   G   A   A   A   T   T   T   A
*   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
T   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
T   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
G   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
G   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
G   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
C   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
G   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
C   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
C   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
G   0   0   0   0   0

[('GGCGCCGGG---TTTA', 'GGC-CCGGGAAATTTA')]